Use GPU

In [1]:
!nvidia-smi

Tue Dec  3 13:30:27 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 552.22         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3050 ...    On  |   00000000:01:00.0  On |                  N/A |
| N/A   44C    P0              4W /   65W |     342MiB /   4096MiB |     86%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import torch
from torch import nn

torch.device('cpu'), torch.cuda.device('cuda'), torch.cuda.device('cuda:1')

(device(type='cpu'),
 <torch.cuda.device at 0x7fe1846aad90>)

In [3]:
torch.cuda.device_count()

1

In [4]:
def try_gpu(i=0):
    """If GPU exists, return gpu(i). Otherwise, return cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():
    """Return all usable GPU. If there is no GPU, return CPU"""
    devices = [
        torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())
    ]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [5]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [6]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [8]:
Y = torch.rand(2, 3, device=try_gpu())
Y

tensor([[0.1527, 0.8378, 0.5614],
        [0.1905, 0.8544, 0.5819]], device='cuda:0')

In [10]:
Z = X.cuda(0)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


In [11]:
Y, Z, Y + Z

(tensor([[0.1527, 0.8378, 0.5614],
         [0.1905, 0.8544, 0.5819]], device='cuda:0'),
 tensor([[1., 1., 1.],
         [1., 1., 1.]], device='cuda:0'),
 tensor([[1.1527, 1.8378, 1.5614],
         [1.1905, 1.8544, 1.5819]], device='cuda:0'))

In [13]:
Z.cuda(0) is Z

True

In [14]:
net = nn.Sequential(
    nn.Linear(3, 1)
)
net = net.to(device=try_gpu())
net(X)

tensor([[0.2586],
        [0.2586]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [15]:
net[0].weight.data.device

device(type='cuda', index=0)